In [1]:
!pip install -q --upgrade s3fs pyarrow scikit-learn xgboost joblib


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [49 lines of output]
      INFO:xgboost.packager.build_wheel:Parsed build configuration: {'hide_cxx_symbols': True, 'use_openmp': True, 'use_cuda': False, 'use_nccl': False, 'use_dlopen_nccl': False, 'plugin_federated': False, 'plugin_rmm': False, 'use_system_libxgboost': False}
      INFO:xgboost.packager.build_wheel:Copying project files to temporary directory /tmp/tmpus6z7o4c/whl_workspace
      INFO:xgboost.packager.build_wheel:Copying /tmp/pip-install-tcoxy_ax/xgboost_67f7f881152e4d52a2daf78e7e96f420/pyproject.toml -> /tmp/tmpus6z7o4c/whl_workspace/pyproject.toml
      INFO:xgboost.packager.build_wheel:Copying /tmp/pip-install-tcoxy_ax/xgboost_67f7f881152e4d52a2daf78e7e96f420/hatch_build.py -> /tmp/tmpus6z7o4c/whl_workspace/hatch_build.py
      INFO:xgboost.packager.build_wheel:Copying /tmp/pip-install-tcoxy_ax/xgboost_67f7f881152e4d52a2daf78e7e96f420/R

In [3]:
import xgboost as xgb
import pandas as pd
import s3fs
import pyarrow as pa

print("xgboost version:", xgb.__version__)
print("✅ All core libraries working fine.")


xgboost version: 3.0.5
✅ All core libraries working fine.


In [ ]:
import pyarrow.dataset as ds

fs = s3fs.S3FileSystem()

dataset = ds.dataset("ys-flight-data-gold/ml-2024/", filesystem=fs, format="parquet")

df = dataset.to_table().to_pandas()

print("Shape:", df.shape)
df.head()


Shape: (1164850, 13)


,flight_date,carrier,operating_carrier,origin,destination,distance,dep_delay_minutes,arr_delay_minutes,is_delayed,month,day_of_week,season,year
0,2024-01-14,UA,G7,MHT,EWR,209.0,71.0,45.0,1,1,7,Winter,2024
1,2024-01-14,UA,G7,IAD,EWR,212.0,0.0,26.0,1,1,7,Winter,2024
2,2024-01-14,UA,G7,EWR,MHT,209.0,74.0,86.0,1,1,7,Winter,2024
3,2024-01-14,UA,G7,STL,ORD,258.0,0.0,25.0,1,1,7,Winter,2024
4,2024-01-14,UA,G7,STL,IAD,696.0,33.0,18.0,1,1,7,Winter,2024


In [ ]:
print("rows,cols:", df.shape)
display(df.dtypes)   


rows,cols: (1164850, 13)


flight_date           object
carrier               object
operating_carrier     object
origin                object
destination           object
distance             float64
dep_delay_minutes    float64
arr_delay_minutes    float64
is_delayed             int32
month                  int32
day_of_week            int32
season                object
year                   int32
dtype: object

In [ ]:
import pandas as pd

features = ['month', 'day_of_week', 'season', 'carrier', 'operating_carrier',
            'origin', 'destination', 'distance', 'dep_delay_minutes']
target = 'is_delayed'

df_model = df[features + [target]].copy()

# fill numeric nulls
df_model['distance'] = df_model['distance'].fillna(df_model['distance'].median())
df_model['dep_delay_minutes'] = df_model['dep_delay_minutes'].fillna(0)

# check nulls
print("Nulls:\n", df_model.isna().sum())


Nulls:
 month                0
day_of_week          0
season               0
carrier              0
operating_carrier    0
origin               0
destination          0
distance             0
dep_delay_minutes    0
is_delayed           0
dtype: int64


In [7]:
from sklearn.preprocessing import LabelEncoder

cat_cols = ['season', 'carrier', 'operating_carrier', 'origin', 'destination']
encoders = {}

for col in cat_cols:
    le = LabelEncoder()
    df_model[col] = le.fit_transform(df_model[col].astype(str))
    encoders[col] = dict(zip(le.classes_, le.transform(le.classes_)))

print("✅ Encoding complete!")
df_model.head()


✅ Encoding complete!


,month,day_of_week,season,carrier,operating_carrier,origin,destination,distance,dep_delay_minutes,is_delayed
0,1,7,0,8,8,215,112,209.0,71.0,1
1,1,7,0,8,8,158,112,212.0,0.0,1
2,1,7,0,8,8,112,215,209.0,74.0,1
3,1,7,0,8,8,319,238,258.0,0.0,1
4,1,7,0,8,8,319,158,696.0,33.0,1


In [8]:
from sklearn.model_selection import train_test_split

X = df_model.drop(columns=[target])
y = df_model[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train shape:", X_train.shape)
print("Target ratio:", y_train.mean())


Train shape: (931880, 9)
Target ratio: 0.22402884491565436


In [9]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

# Handle imbalance
scale_pos_weight = (len(y_train) - y_train.sum()) / y_train.sum()

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_test, label=y_test)

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'eta': 0.1,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'scale_pos_weight': scale_pos_weight,
    'verbosity': 1
}

watchlist = [(dtrain, 'train'), (dval, 'eval')]
bst = xgb.train(params, dtrain, num_boost_round=200, evals=watchlist, early_stopping_rounds=20)


[0]	train-auc:0.57995	eval-auc:0.57694
[1]	train-auc:0.92071	eval-auc:0.92032
[2]	train-auc:0.92296	eval-auc:0.92266
[3]	train-auc:0.92327	eval-auc:0.92310
[4]	train-auc:0.92356	eval-auc:0.92343
[5]	train-auc:0.92424	eval-auc:0.92406
[6]	train-auc:0.92447	eval-auc:0.92419
[7]	train-auc:0.92448	eval-auc:0.92426
[8]	train-auc:0.92453	eval-auc:0.92433
[9]	train-auc:0.92486	eval-auc:0.92460
[10]	train-auc:0.92518	eval-auc:0.92492
[11]	train-auc:0.92545	eval-auc:0.92520
[12]	train-auc:0.92552	eval-auc:0.92527
[13]	train-auc:0.92560	eval-auc:0.92539
[14]	train-auc:0.92579	eval-auc:0.92552
[15]	train-auc:0.92581	eval-auc:0.92549
[16]	train-auc:0.92615	eval-auc:0.92584
[17]	train-auc:0.92623	eval-auc:0.92592
[18]	train-auc:0.92663	eval-auc:0.92634
[19]	train-auc:0.92661	eval-auc:0.92631
[20]	train-auc:0.92662	eval-auc:0.92631
[21]	train-auc:0.92665	eval-auc:0.92635
[22]	train-auc:0.92670	eval-auc:0.92644
[23]	train-auc:0.92708	eval-auc:0.92674
[24]	train-auc:0.92705	eval-auc:0.92669
[25]	train

In [10]:
y_prob = bst.predict(dval)
y_pred = (y_prob > 0.5).astype(int)

print("AUC:", roc_auc_score(y_test, y_prob))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


AUC: 0.9368635245106276
              precision    recall  f1-score   support

           0       0.95      0.93      0.94    180778
           1       0.77      0.82      0.79     52192

    accuracy                           0.91    232970
   macro avg       0.86      0.87      0.87    232970
weighted avg       0.91      0.91      0.91    232970

[[168246  12532]
 [  9529  42663]]


In [11]:
import pandas as pd

fi = pd.Series(bst.get_score(importance_type='gain')).sort_values(ascending=False)
print(fi.head(10))
fi.to_csv("feature_importance.csv")


dep_delay_minutes    2608.307373
carrier                47.486359
operating_carrier      41.945309
destination            39.463253
day_of_week            39.117653
origin                 35.456066
distance               34.088089
dtype: float64


In [12]:
import joblib, boto3, os

local_dir = "/home/ec2-user/SageMaker/artifacts"
os.makedirs(local_dir, exist_ok=True)

bst.save_model(f"{local_dir}/xgb_flight_delay.json")
joblib.dump(encoders, f"{local_dir}/label_encoders.pkl")

s3 = boto3.client("s3")
bucket = "ys-flight-data-gold"

s3.upload_file(f"{local_dir}/xgb_flight_delay.json", bucket, "models/xgb_flight_delay.json")
s3.upload_file(f"{local_dir}/label_encoders.pkl", bucket, "models/label_encoders.pkl")
s3.upload_file("feature_importance.csv", bucket, "models/feature_importance.csv")

print(f"✅ Model + encoders saved to s3://{bucket}/models/")


✅ Model + encoders saved to s3://ys-flight-data-gold/models/
